In [35]:
import pandas as pd
import numpy as np
df = pd.read_csv("Happiness_Report_2012_2020.csv")
df.head()

# Any country that has a zero in any of the columns is not being compared on an equal footing, so remove it from the dataset
df_clean = df.replace({'0':np.nan, 0:np.nan})
df_clean = df_clean.dropna(how="any")
print(f"There were {len(df)} data points in the original Dataframe, but only {len(df_clean)} usable points")

# Define the year values to loop through
years = [2012, 2013, 2015, 2016, 2017, 2018, 2019, 2020]

# Create a series of the of the countries to include in the analysis
countries = df_clean.Country.unique()

# Get the number of countries for future reference
country_counts = df_clean.Country.value_counts()

# Exclude countries that are not present in all years of the report
clean_countries = []
for country in countries:
    # If country has seven occurences, add it to clean_countries
    if country_counts[f"{country}"] == len(years):
        clean_countries.append(country)

print(f"There were {len(countries)} in the clean dataset, but only {len(clean_countries)} countries that appeared in every report with clean data")

# Create a new dataframe that considers only the countries that are there every year
df_clean = df.loc[df['Country'].isin(clean_countries)]

There were 1247 data points in the original Dataframe, but only 1247 usable points
There were 168 in the clean dataset, but only 139 countries that appeared in every report with clean data


In [36]:
# Add a Normalized Happiness Score Column to the dataset
df_clean["Normalized Happiness Score"] = np.nan

for country in clean_countries:
    scores = df_clean.loc[df_clean["Country"] == country, "Happiness Score"]
    scores_normalized = scores / max(scores)

    for i in range(len(scores_normalized)):

        df_clean.at[scores_normalized.index[i], "Normalized Happiness Score"] = scores_normalized.iloc[i]

C:\Users\hawki\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
df_clean.to_csv("Augmented Repoert.csv")

In [38]:
loc = df_clean.loc[df_clean["Country"] == country]
loc["Report Year"].to_list()

[2015, 2016, 2017, 2018, 2019, 2020, 2013, 2012]

In [44]:
from scipy import stats

summary_countries = []
summary_slopes = []

# Create a summary Dataframe With Country Names and the Slope of the Trendline for Year of the report and Normalized Happiness
for country in clean_countries:
    loc = df_clean.loc[df_clean["Country"] == country]
    x = loc["Report Year"].to_list()
    # 2012 gathered data for 2005 - 2011
    # 2013 took into account data for 2010 - 2012
    # 2015 yook into account data for 2012 - 2014
    # 2016 took into account data for 2013 - 2015
    # 2017 2014 - 2016
    # 2018 2015 -2017
    # 2019 2016 - 2018
    # 2020 2017 - 2019
    x[x.index(2012)] = 2008
    x[x.index(2013)] = 2011
    x[x.index(2015)] = 2013
    x[x.index(2016)] = 2014
    x[x.index(2017)] = 2015
    x[x.index(2018)] = 2016
    x[x.index(2019)] = 2017
    x[x.index(2020)] = 2018
    
    y = loc["Normalized Happiness Score"].to_list()
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    summary_countries.append(country)
    summary_slopes.append(slope * 100)

summary_df = pd.DataFrame(data = {"Country" : summary_countries, "Percent Change": summary_slopes})

In [45]:
summary_df.to_csv("Change_Summary.csv")
summary_df

,Country,Percent Change
0,Switzerland,-0.048765
1,Iceland,0.721234
2,Denmark,-0.269255
3,Norway,-0.055772
4,Canada,-0.354820
...,...,...
134,Afghanistan,-3.169081
135,Rwanda,-1.949547
136,Benin,3.180770
137,Burundi,-0.326169


In [46]:
# For any country in the Tableau country list that does not have a data point, set the change to 0
all_countries = pd.read_csv("Tableau_country_list.csv")

scored_countries = summary_df["Country"].to_list()
print(scored_countries)
# for each country in the the master country list,
for i in range(len(all_countries)):
    
    # Check if the country is in the existing dataframe. If it is, do nothing
    country = all_countries.iloc[i]["Country"]
    
    # If it is NOT, add it to the df with value of zero
    if not (country in scored_countries):
        summary_df = summary_df.append({'Country': country, "Percent Change": 0}, ignore_index=True)
summary_df

['Switzerland', 'Iceland', 'Denmark', 'Norway', 'Canada', 'Finland', 'Netherlands', 'Sweden', 'New Zealand', 'Australia', 'Israel', 'Costa Rica', 'Austria', 'Mexico', 'United States', 'Brazil', 'Luxembourg', 'Ireland', 'Belgium', 'United Arab Emirates', 'United Kingdom', 'Venezuela', 'Singapore', 'Panama', 'Germany', 'Chile', 'France', 'Argentina', 'Czech Republic', 'Uruguay', 'Colombia', 'Thailand', 'Saudi Arabia', 'Spain', 'Malta', 'Taiwan', 'Kuwait', 'Trinidad and Tobago', 'El Salvador', 'Guatemala', 'Uzbekistan', 'Slovakia', 'Japan', 'South Korea', 'Ecuador', 'Bahrain', 'Italy', 'Bolivia', 'Moldova', 'Paraguay', 'Kazakhstan', 'Slovenia', 'Lithuania', 'Nicaragua', 'Peru', 'Belarus', 'Poland', 'Malaysia', 'Croatia', 'Libya', 'Russia', 'Jamaica', 'Cyprus', 'Algeria', 'Kosovo', 'Turkmenistan', 'Mauritius', 'Hong Kong', 'Estonia', 'Indonesia', 'Vietnam', 'Turkey', 'Kyrgyzstan', 'Nigeria', 'Azerbaijan', 'Pakistan', 'Jordan', 'Montenegro', 'China', 'Zambia', 'Romania', 'Serbia', 'Portugal

,Country,Percent Change
0,Switzerland,-0.048765
1,Iceland,0.721234
2,Denmark,-0.269255
3,Norway,-0.055772
4,Canada,-0.354820
...,...,...
475,"Bahamas, The",0.000000
476,Barbados,0.000000
477,Belize,0.000000
478,Bermuda,0.000000


In [47]:
summary_df.to_csv("Change_Summary_Tableau.csv")


In [7]:
# Create an empty df with columns "Country" "Report Year" "Position Change"
yoy_df = pd.DataFrame(columns = ['Country', 'Report Year', "Clean Rank", "Position Change", "Happiness Change"])

# grab only certain columns, and one year at a time. Sort it by 
dfs = []

for year in years:
    
    rows_df = df_clean.loc[df['Report Year'] == year, df_clean.columns.isin(["Country", "Report Year", "Overall Rank", "Happiness Score"])].sort_values(by=['Overall Rank'])
    rows_df["Clean Rank"] = np.arange(0, len(df_clean.loc[df['Report Year'] == year]))
    dfs.append(rows_df)

In [8]:
print(dfs[0].loc[dfs[0]["Country"] == "Norway", "Clean Rank"])
print(dfs[1].loc[dfs[1]["Country"] == "Norway", "Clean Rank"])
print(dfs[2].loc[dfs[2]["Country"] == "Norway", "Clean Rank"])
print(dfs[3].loc[dfs[3]["Country"] == "Norway", "Clean Rank"])
print(dfs[4].loc[dfs[4]["Country"] == "Norway", "Clean Rank"])
print(dfs[5].loc[dfs[5]["Country"] == "Norway", "Clean Rank"])

1093    2
Name: Clean Rank, dtype: int32
936    1
Name: Clean Rank, dtype: int32
3    3
Name: Clean Rank, dtype: int32
161    3
Name: Clean Rank, dtype: int32
315    0
Name: Clean Rank, dtype: int32
471    1
Name: Clean Rank, dtype: int32


In [9]:
for i in range(len(dfs) - 1):
    
    d_position = []
    d_happiness = []
    
    for row in range(len(dfs[i])):
        
        country = dfs[i+1].iloc[row]["Country"]
        # Get the data for a given country at the given year as well as the previous year
        last_report = dfs[i].loc[dfs[i]["Country"] == f"{country}"]
        new_report = dfs[i+1].loc[dfs[i+1]["Country"] == f"{country}"]

        # Get only the rank and the overall happiness score
        idx_new = new_report.index.to_list()
        idx_last = last_report.index.to_list()

        if len(idx_new) > 1:
            print(f"Error: Multiple reports recieved for {country} in {years[i+1]}")
                                                                       
        elif len(idx_last) >1 :
            print(f"Error: Multiple reports recieved for {country} in {years[i]}")

        else:
            new_pos = new_report.at[idx_new[0],"Clean Rank"]
            last_pos = last_report.at[idx_last[0],"Clean Rank"]
            new_happiness = new_report.at[idx_new[0],"Happiness Score"]
            last_happiness = last_report.at[idx_last[0],"Happiness Score"]

        d_position.append(last_pos - new_pos)
        d_happiness.append(new_happiness - last_happiness)

    dfs[i+1]["Position Change"] = d_position
    dfs[i+1]["Happiness Change"] = d_happiness
    #dfs[i+1] = dfs[i+1].astype({'Position Change': 'int32'})
                                                                       
    yoy_df = pd.concat([yoy_df, dfs[i+1]], ignore_index=True)

yoy_df = yoy_df.astype({'Position Change': 'int32'})
yoy_df = yoy_df.astype({'Clean Rank': 'int32'})



C:\Users\hawki\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [10]:
yoy_df.loc[yoy_df["Country"] == "Norway"]

,Clean Rank,Country,Happiness Change,Happiness Score,Overall Rank,Position Change,Report Year
1,1,Norway,0.125588,7.655,2.0,1,2013
142,3,Norway,-0.133000,7.522,4.0,-2,2015
281,3,Norway,-0.024000,7.498,4.0,0,2016
417,0,Norway,0.039000,7.537,1.0,3,2017
557,1,Norway,0.057000,7.594,2.0,-1,2018
697,2,Norway,-0.040000,7.554,3.0,-1,2019
838,4,Norway,-0.066000,7.488,5.0,-2,2020


In [11]:
summary_df = yoy_df.groupby(["Country"]).mean()

In [12]:
print(summary_df.sort_values(by="Position Change", ascending=False))

                     Clean Rank  Happiness Change  Happiness Score  \
Country                                                              
Benin                120.000000          0.247073         4.035571   
Congo (Brazzaville)  107.571429          0.197656         4.482629   
Philippines           71.000000          0.169258         5.418286   
Serbia                75.428571          0.172948         5.326743   
Latvia                62.142857          0.170463         5.625286   
...                         ...               ...              ...   
India                114.571429         -0.197192         4.262043   
Myanmar              113.428571         -0.145215         4.380286   
Turkmenistan          67.428571         -0.203154         5.522586   
Jordan                82.571429         -0.155493         5.135057   
Venezuela             63.857143         -0.262988         5.678457   

                     Overall Rank  Position Change  
Country                             

In [13]:
print(summary_df.sort_values(by="Happiness Change", ascending=False))

                     Clean Rank  Happiness Change  Happiness Score  \
Country                                                              
Benin                120.000000          0.247073         4.035571   
Congo (Brazzaville)  107.571429          0.197656         4.482629   
Serbia                75.428571          0.172948         5.326743   
Bulgaria             105.714286          0.172801         4.596500   
Latvia                62.142857          0.170463         5.625286   
...                         ...               ...              ...   
Botswana             127.571429         -0.155524         3.799843   
India                114.571429         -0.197192         4.262043   
Turkmenistan          67.428571         -0.203154         5.522586   
Afghanistan          133.000000         -0.239478         3.452986   
Venezuela             63.857143         -0.262988         5.678457   

                     Overall Rank  Position Change  
Country                             

In [15]:
yoy_df.to_csv("Analysis.csv")

In [41]:
# Add a Normalized Happiness Score Column to the dataset
df["Normalized Happiness Score"] = np.nan
scores = df.loc[df["Country"] == "Benin", "Happiness Score"]
scores_normalized = scores / max(scores)

In [42]:
for i in range(len(scores_normalized)):
    
    df.at[scores_normalized.index[i], "Normalized Happiness Score"] = scores_normalized.iloc[i]


In [43]:
df.loc[df["Country"] == "Benin"]

,Report Year,Overall Rank,Country,Happiness Score,Normalized Happiness Score
154,2015,155,Benin,3.340000,0.640337
310,2016,153,Benin,3.484000,0.667945
457,2017,143,Benin,3.657000,0.701112
605,2018,136,Benin,4.141000,0.793903
727,2019,102,Benin,4.883000,0.936158
867,2020,86,Benin,5.216000,1.000000
1089,2013,155,Benin,3.528000,0.676380
1245,2012,155,Benin,3.486486,0.668421


# Clean the Data

In [48]:
import json

# Create a list of countries from the Geojson
# Load existing data
with open('resources/countries.geojson') as f:
  geodata = json.load(f)

geojson_countries =[]

for datum in geodata["features"]:

    geojson_countries.append(datum["properties"]["ADMIN"])


# Crate a list of countries from the analyzed data
import pandas as pd
import numpy as np
df = pd.read_csv("Change_summary_Tableau.csv")
df.head()
countries = df.Country.unique()

# Show any country that is in the raw data, but not found in the geojson
for country in countries:
    
    if country in geojson_countries:
        pass
    else:
        print(country)

United States
Hong Kong
Serbia
Palestinian Territories
Congo (Kinshasa)
Congo (Brazzaville)
Tanzania
Cote d'Ivoire
Gambia, The
Guinea-Bissau
Hong Kong SAR
Macao SAR
Timor-Leste
Faeroe Islands
North Macedonia
Micronesia, Fed. Sts.
Bahamas, The
Curacao
Sint Maarten (Dutch part)
St. Kitts and Nevis
St. Lucia
St. Martin (French part)
St. Vincent and the Grenadines


## Manually Clean the geojson
"United States of America" = "United States" 
"Hong Kong S.A.R" = "Hong Kong"
"Republic of Serbia" = "Serbia" 
"Palestine" = "Palestinian Territories" 
"Democratuc Republic of the Congo" = "Congo (Kinshasa)" 
"Republic of Congo" = "Congo (Brazzaville)"
"United Republic of Tanzania" = "Tazania"

# Create a custom GeoJson with only the analyzed countries

In [50]:
import json
import pandas as pd


data = pd.read_csv("Change_Summary_Tableau.csv")
data_analyzed = pd.read_csv("Change_Summary.csv")
countries = data.Country.to_list()
happiness = data["Percent Change"].to_list()

countries_analyzed = data_analyzed.Country.to_list()



# Load existing data
with open('resources/countries_rename.geojson') as f:
  geodata = json.load(f)

countries_validate =[]
new_features = []
for i in range(len(countries)):
    
    for datum in geodata["features"]:

        country = datum["properties"]["ADMIN"]
        if country == countries[i]:
            countries_validate.append(country)
            properties = datum["properties"]
            # if the country exists in the full analysis, put the number, else put 999
            if country in countries_analyzed:
                properties.update({'happiness':happiness[i]})
            else:
                properties.update({'happiness':999})

            datum.update({"properties":properties})
            new_features.append(datum)

new_json = {
            "type": "Feature Collection",
            "features": new_features
           }

# Write JSON file with new data
with open('resources/countries_happiness.geojson', 'w') as f:
  f.write(json.dumps(new_json))

# Check for any country in the analysis that is not in the json
for country in countries:

    if country in countries_validate:
        pass
    else:
        print(country)

Hong Kong SAR
Macao SAR
Timor-Leste
Faeroe Islands
Micronesia, Fed. Sts.
Bahamas, The
Curacao
Sint Maarten (Dutch part)
St. Kitts and Nevis
St. Lucia
St. Martin (French part)
St. Vincent and the Grenadines
Hong Kong SAR
Macao SAR
Timor-Leste
Faeroe Islands
Micronesia, Fed. Sts.
Bahamas, The
Curacao
Sint Maarten (Dutch part)
St. Kitts and Nevis
St. Lucia
St. Martin (French part)
St. Vincent and the Grenadines
Hong Kong SAR
Macao SAR
Timor-Leste
Faeroe Islands
Micronesia, Fed. Sts.
Bahamas, The
Curacao
Sint Maarten (Dutch part)
St. Kitts and Nevis
St. Lucia
St. Martin (French part)
St. Vincent and the Grenadines
Hong Kong SAR
Macao SAR
Timor-Leste
Faeroe Islands
Micronesia, Fed. Sts.
Bahamas, The
Curacao
Sint Maarten (Dutch part)
St. Kitts and Nevis
St. Lucia
St. Martin (French part)
St. Vincent and the Grenadines
Hong Kong SAR
Macao SAR
Timor-Leste
Faeroe Islands
Micronesia, Fed. Sts.
Bahamas, The
